In [1]:
import pickle
import json
import os
import math
from gensim.models import Word2Vec

In [3]:
data = []
for i in range(100):
    with open('processed/sentences/sen'+str(i), 'rb') as f:
        data += pickle.load(f)
len(data)

0

In [ ]:
with open('processed/entities.pkl', 'rb') as f:
    entities = pickle.load(f)

## 关系标注

In [ ]:
# 根据规则 替换所有关系

tgs = {
    "per": ["人物", "歌手", "演员", "作家"],
    "org": ["机构", "企业", "公司", "学校", "部门", "大学"],
    "pl": ["地点", "地理", "城市", "国家", "地区"]
}

config = {
        # person  9
    "per2per_family_members" : ["父亲","母亲","丈夫","妻子","儿子","女儿","哥哥","妹妹","姐姐","弟弟","孙子",
                        "孙女","爷爷","奶奶","外婆", "外公","家人","家庭成员" ,"夫人","对象","夫君"],
    "per2per_social_members" : ["朋友", "好友", "同学", "合作", "搭档", "经纪人", "师从"],

    "per2pl_birth_place" : ["出生地", "出生于", "来自", "歌手出生地", "作者出生地", "出生在", "作者出生地", "出生"],
    "per2pl_live_place" : ["居住地", "主要居住地", "居住", "现居住", "目前居住地", "现居住于", "居住地点", "居住于"],
    
    "per2pl_country": ["国籍", "国家"],
    "per2org_graduate_from" : ["毕业院校", "毕业于", "毕业学院", "本科毕业院校", "最后毕业院校", "毕业高中", "毕业地点", "本科毕业学校", "知名校友"],
    "per2org_belong_to" : ["隶属单位", "经纪公司", "隶属关系", "行政隶属", "隶属学校", "隶属大学", "隶属地区", "所属公司", "签约公司", "任职公司", "工作单位", "所属"],

    "per2oth_profession" : ['职业'],
    "per2oth_nation" : ['民族'],

    # orgnazition  9
    "org2per_owner" : ["拥有", "拥有者"],
    "org2per_founder" : ["创始人", "创始", "主要创始人", "集团创始人"],
    "org2per_school_leader" : ["校长", "现任校长", "学校校长", "总校长"],
    "org2per_leader" : ["领导", "现任领导", "领导单位", "主要领导", "领导人", "主要领导人"],

    "org2org_surroundings" : ["周围景观", "周边景点"],


    "org2pl_location" : ["所属地区","国家", "地区", "地理位置", "位于", "区域", "地点", "总部地点", "所在地", "所在区域", "位于城市", "总部位于", "酒店位于", "学校位于", "最早位于", "地址", "所在城市", "城市", "主要城市", "坐落于"],


    # place  7
    "pl2per_main_character" : ["相关人物", "知名人物", "历史人物"],

    "pl2pl_location" : ["所属地区","所属国", "所属洲", "所属州", "所属国家", "最大城市", "地区", "地理位置", "位于", "区域", "地点", "总部地点", "所在地", "所在区域", "位于城市", "总部位于", "酒店位于", "学校位于", "最早位于", "地址", "所在城市", "城市", "主要城市", "坐落于"],
    "pl2pl_adjacement" : ["毗邻", "东邻", "邻近行政区", "相邻", "紧邻", "邻近", "北邻", "南邻", "邻国"],
    "pl2pl_contains" : ["包含", "包含国家", "包含人物", "下辖地区", "下属"],
    "pl2pl_captial" : ["首都"],

    "pl2org_sights" : ["著名景点", "主要景点", "旅游景点", "特色景点"],
    "pl2oth_climate" : ["气候类型", "气候条件", "气候", "气候带"],
}


name = {
    "per2per_family_members": "/人物/人物/家庭成员",
    "per2per_social_members": "/人物/人物/社交关系",

    "per2pl_birth_place": "/人物/地点/出生地",
    "per2pl_live_place": "/人物/地点/居住地",
    "per2pl_country" : "/人物/地点/国籍",
    "per2org_graduate_from": "/人物/组织/毕业于",
    "per2org_belong_to": "/人物/组织/属于",

    "per2oth_profession": "/人物/其它/职业",
    "per2oth_nation": "/人物/其它/民族",

    "org2per_owner": "/组织/人物/拥有者",
    "org2per_founder": "/组织/人物/创始人",
    "org2per_school_leader": "/组织/人物/校长",
    "org2per_leader": "/组织/人物/领导人",

    "org2org_surroundings": "/组织/组织/周边",


    "org2pl_location": "/组织/地点/位于",


    "pl2per_main_character": "/地点/人物/相关人物",

    "pl2pl_location": "/地点/地点/位于",
    "pl2pl_adjacement": "/地点/地点/毗邻",
    "pl2pl_contains": "/地点/地点/包含",
    "pl2pl_captial": "/地点/地点/首都",

    "pl2org_sights": "/地点/组织/景点",
    "pl2oth_climate": "/地点/其它/气候"
}

In [ ]:
def check(string, tgs):
    for t in tgs:
        if t in string:
            return True
    return False

In [ ]:
# 标注
processed_data = []
for can in data:
    # can  [sentence, head, tail, segment]
    sentence, head, tail, segment = can
    if tail not in entities[can[1]].values():
        can.append('NA')
        processed_data.append(can)
        continue
    rel = ''
    for key, value in entities[can[1]].items():
        if value==tail:
            rel = key
    
    for key, value in config.items():
        if rel in value:
            tp = key.split('_')[0].split('2')
            if check(entities[can[1]].get('BaiduTAG', ""), tgs[tp[0]]):
                if tp[1]=='oth':
                    can.append(name[key])
                    processed_data.append(can)
                elif check(entities[can[2]].get('BaiduTAG', ""), tgs[tp[1]]):
                    can.append(name[key])
                    processed_data.append(can)
            break
len(processed_data)

In [ ]:
e2id = {}
count = 0
e_set = set()
for i in processed_data:
    e_set.add(i[1])
    e_set.add(i[2])
for e in e_set:
    e2id[e] = count
    count += 1

In [ ]:
total_data = []
for d in processed_data:
    total_data.append({
        'head':{
            'word': d[1],
            'id': str(e2id[d[1]])
        },
        'relation': d[-1],
        'tail': {
            'word': d[2],
            'id': str(e2id[d[2]])
        },
        'sentence': ' '.join(d[-2]),
        'ori_sen': d[0],
        'sen_seg': d[-2]
    })

In [ ]:
rl = {}
for i in total_data:
    rl[i['relation']] = rl.get(i['relation'], 0) + 1

trl = {}
record = {}
for k,v in rl.items():
    trl[k] = int(max(math.floor(v*0.25), 1))
    record[k] = 0

In [ ]:
train = []
test = []
for i in total_data:
    if record[i['relation']]<=trl[i['relation']]:
        test.append(i)
        record[i['relation']] += 1
    else:
        train.append(i)

In [ ]:
if not os.path.isdir('../data'):
    os.mkdir('../data')
if not os.path.isdir('../data/chinese'):
    os.mkdir('../data/chinese')

In [ ]:
with open('../data/chinese/train.json', 'w', encoding='utf8') as f:
    json.dump(train, f)
with open('../data/chinese/test.json', 'w', encoding='utf8') as f:
    json.dump(test, f)

In [ ]:
count = 0
r2id = {}
for k in list(rl.keys()):
    r2id[k] = count
    count +=1
with open('../data/chinese/rel2id.json', 'w', encoding='utf8') as f:
    json.dump(r2id, f)

## 训练词向量

In [ ]:
senlist = []
for d in data:
    senlist.append(d[-1])

In [ ]:
model = Word2Vec(senlist, sg=5, min_count=1, size=50, workers=4)

In [ ]:
w2v = {}
for i in model.wv.index2word:
    w2v[i] = model[i]
len(w2v)

In [ ]:
new_w2v = []
for word, vec in w2v.items():
    new_w2v.append({
        'word': word,
        'vec': [float(i) for i in vec]
    })

In [ ]:
with open('../data/chinese/word_vec.json', 'w', encoding='utf8') as f:
    json.dump(new_w2v, f)